One off script to take a dozen pickle files of usage data and turn them into one file

In [2]:


import pickle
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
from collections import defaultdict

import sys
sys.path.append("..")
from clustering import make_usage_matrices


/home/gsc685/.conda/envs/gabriella_cwr4lsc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
decades = [decade for decade in np.arange(1910, 2009, 10)]
out_path = '../data/cwr4lsc/binder/usages_16_len128.dict'

all_usages = defaultdict(list)

for decade in decades:
    path = '../data/cwr4lsc/binder/usages_with_vectors_16_len128_{}.dict'.format(decade)
    with open(path, 'rb') as f:
        decade = pickle.load(f)
        
        #print(decade['leaf'])[0]

        # remove annoying initial (word, vector) tuple to just vector
        for target in decade.keys():
            decade[target] = [(vec, x, y, z) for vec, x, y, z in decade[target] ] 
            #decade[target] = (big_tuple[0][1], big_tuple[1], big_tuple[2], big_tuple[3]) for big_tuple in decade[target]


        
        #raise Exception('not done')

        for target in decade.keys():
            all_usages[target] += (decade[target])

In [6]:
len(all_usages['parent'])

6285

In [7]:
# how many dimensions do our vectors have?

In [8]:
all_usages['leaf'][0]

(array([ 5.6224475e+00,  2.8358703e+00,  1.3824366e+00,  5.3413668e+00,
         4.2947555e+00,  8.2244134e-01,  3.0668213e+00,  1.3716707e+00,
         1.1286278e+00,  1.0092518e+00,  1.2194594e+00,  5.4043293e+00,
         2.8424539e-02,  3.5405785e-01,  4.5536084e+00,  2.4813056e+00,
         4.5271816e+00,  4.4619012e+00, -1.7319861e-01,  8.3986789e-01,
         5.7031310e-01,  4.4277871e-01,  6.6316170e-01,  1.4368700e+00,
         4.3249574e-01, -2.4854717e-01,  9.2216694e-01,  1.5625881e+00,
         6.0887563e-01,  1.8470562e+00,  1.0632853e-01,  2.9497525e-01,
         9.3153203e-01,  1.9187591e+00,  2.7344732e+00,  9.1944015e-01,
         1.0580186e+00,  4.5944220e-01,  1.2401076e-02, -2.0799415e-01,
        -5.3534284e-03, -1.3480511e-01, -6.6288763e-01, -1.4357696e+00,
        -6.2934172e-01,  4.0593222e-02,  3.2758760e-01, -7.1864498e-01,
         1.5988810e+00,  2.9125583e-01,  2.6643238e+00, -8.3092999e-01,
         1.5239185e+00, -7.0998663e-01, -7.5125724e-01, -5.08025

In [11]:
# save into another file with all usages, in the format tht the clusterizer expects


with open(out_path, 'wb') as f:
    pickle.dump(all_usages, f)

now we have the right number of things to unpack in a single usage.

In [12]:
len(all_usages['leaf'][0])

4

In [13]:
with open(out_path, 'rb') as f:
    usages_test = pickle.load(f)

In [14]:
for w in usages_test:
    _, _, _, _ = usages_test

ValueError: too many values to unpack (expected 4)

actually it still doesn't work. the function doesn't want a llist of individual usages for a word, it wants a matrix for each word

We have to make udsage matrices actually which is what the clusterizer really expects. there's a utility for this already but we have to do it ourselves

In [16]:
in_path = '../data/cwr4lsc/binder/usages_16_len128.dict'
out_path = '../data/cwr4lsc/binder/matrix_usages_16_len128.dict'

matrix_usages = make_usage_matrices(in_path, usages_out=None, ndims=62) # make sure to use the ndims of the feature prediction model
with open(out_path, 'wb') as f:
    pickle.dump(matrix_usages, f)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [01:39<00:00,  5.26s/it]
